<a href="https://colab.research.google.com/github/ranaurek/Quantum-Graph-Coloring/blob/main/Classical_looking_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install qiskit

In [ ]:
pip install pylatexenc

In [ ]:
# Preamble
import qiskit
qiskit.__version__
from qiskit import *

from qiskit.visualization import plot_bloch_multivector, plot_histogram
import matplotlib.pyplot as plt


import numpy as np
from math import pi

%matplotlib inline

from IPython.display import Image ## For displaying Images

A Classical-looking Algorithm

In [ ]:
def find_colorings(graph, nNodes, nColors, nShots):

  nc = nColors + 1
  # Calculate how many qubits we need.
  nn2=round((nNodes-1)*nNodes/2) # Number of pairs of different nodes (n choose 2)
  # and of possible edges
  sc=round(nc*nNodes) # Pointer to list of same-color qubits
  # for pairs and edges
  sg=round(nc*nNodes + nn2) # Pointer to list of edge qubits
  nqbits=sc + 2*nn2 # Total number of qubits

  # Create a quantum circuit.
  q = QuantumRegister(nqbits)
  c=ClassicalRegister(nColors*nNodes)
  qc = QuantumCircuit(q,c)

  # Set the edge qubits in the circuit.
  for src in graph.keys():
    if src not in graph:
      continue
    else:
      for dst in graph[src]:
        # edge qubits for n+1 nodes arranged like:
        # 0-1, 0-2, ..., 0-n, 1-2, 1-3, ..., (n-1)-n
        # so to calculate position of edge qubit for src and dst,
        # get pointer to start of 0-i qubits by calculating row offset and then
        # subtracting the number of missing symmetric (redundant) qubits,
        # then offset by (dst-src-1) to get the particular edge between them.
        qc.x(sg + src*nNodes - round(src*(src+1)/2) + (dst - src - 1))

  # Superposition the color qubits for each node.
  s=0
  for n in range(nNodes):
    s=nc*n
    for k in range(nColors):
      qc.h(s+k)

  # Check the constraints to limit superpositions to possible colorings using
  # the ancillary qubits.
  s=0
  for n in range(nNodes):

    s=n*nc

    # Eliminate 11
    for k in range(nColors-1):
      for l in range(k+1,nColors):
        qc.ccx (s+k,s+l,s+nColors)
        qc.cx (s+nColors,s+k)
        qc.reset(s+nColors)

    # Eliminate 0* (no colour assigned to the node n) 
    for k in range(nColors):
      qc.x(s+k)
    cb=list(range(s,s+nColors))
    qc.mcx (cb,s+nColors)
    for k in range(nColors):
      qc.x(s+k)
      qc.cx (s+nColors,s+nColors-1) # NOTE THIS PART OF THE CODE MAKES 001001001 combination the most likely !!! (in 3 nodes 3 colors)
      qc.reset(s+nColors)

  # Set the same-color qubits so that we can check the coloring against the edges.
  for k in range(nColors):
    s=nc*nNodes
    for n1 in range(nNodes-1):
      for n2 in range(n1+1,nNodes):
        n11=nc*n1+k 
        # If q[n11]=|1> it means the node n1 has the color k
        n22=nc*n2+k 
        # If q[n22]=|1> it means the node n2 has the color k
        qc.ccx(n11,n22,s) 
        s# If same color k, set s to |1>.
        # Notice it can happens at most for one k
        s=s+1
        

  # Check that the coloring is valid by comparing same-color and edge qubits.
  for n in range(sc,sc+nn2): # For each pair of nodes
    # If same color and there is an edge 
    #"destroy" (set to |0*>) the colouring
    for node in range(nNodes):
      qnode=nc*node
      qnc=qnode+nColors
      for k in range(nColors):
        cb=[n,n+nn2,qnode+k]
        qc.mcx (cb,qnc)
        cb=[n,n+nn2,qnc]
        qc.mcx (cb,qnode+k)
        qc.reset(qnc)

  # Measure (qiskit notation is weird so account for that)
  cb=nColors*nNodes-1
  for n in range(nNodes):
    s=n*nc
    for k in range(nColors):
      qb=s+k
      qc.measure(qb,cb)
      cb=cb-1

  # Execute the circuit on the qasm simulator.
  backend_sim = Aer.get_backend('qasm_simulator')
  job = execute(qc, backend_sim, shots=nShots)
  # Grab the results from the job.
  result_sim = job.result()
  counts = result_sim.get_counts(qc)

  # Return results.
  return counts

def niceify(counts):
  nice_counts = {}
  for k,v in counts.items():
    nice_counts[",".join([k[nColors * i : nColors * i + nColors] for i in range(len(k) // nColors + 1)])] = v
  return nice_counts

2 Coloring for a 3 Node Graph (arranegd in a line)

In [ ]:
nNodes=3
nColors=2
graph = {
    0 : [1],
    1 : [2]
}

counts = find_colorings(graph, nNodes, nColors, 100)
print(niceify(counts))

{'10,01,10,': 2, '01,10,01,': 14, '00,00,00,': 84}


3 Coloring of a 3 Node Graph (arranged in a cycle)

In [ ]:
nNodes=3
nColors=3
graph = {
    0 : [1, 2],
    1 : [2]
}

counts = find_colorings(graph, nNodes, nColors, 1000)
print(niceify(counts))
#qc.draw()

{'100,001,010,': 15, '001,100,010,': 15, '010,001,100,': 17, '000,000,000,': 898, '001,010,100,': 22, '100,010,001,': 17, '010,100,001,': 16}


2 Coloring of a 3 Node Graph (arranged in a cycle)

In [ ]:
nNodes=3
nColors=2
graph = {
    0 : [1, 2],
    1 : [2]
}

counts = find_colorings(graph, nNodes, nColors, 1000)
print(niceify(counts))

{'00,00,00,': 1000}


2 Coloring of a 4 Node Graph (arranged in a cycle)

In [ ]:
nNodes=4
nColors=2
graph = {
    0 : [1, 3],
    1 : [2],
    2:  [3]
}

counts = find_colorings(graph, nNodes, nColors, 100)
print(niceify(counts))

{'10,01,10,01,': 2, '00,00,00,00,': 94, '01,10,01,10,': 4}


3 Coloring of a 4 Node Graph (run not enough times so no solutions found)

In [ ]:
nNodes=4
nColors=3
graph = {
    0 : [1, 2, 3],
    1 : [3],
    2:  [3]
}

counts = find_colorings(graph, nNodes, nColors, 1)
print(niceify(counts))

{'000,000,000,000,': 1}
